In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,514763
1,Huelva,Confirmados PDIA,73474
2,Huelva,Confirmados PDIA 14 días,5586
3,Huelva,Tasa PDIA 14 días,"1085,1595782913691"
4,Huelva,Confirmados PDIA 7 días,1833
5,Huelva,Tasa PDIA 7 dias,"356,08619889152874"
6,Huelva,Total Confirmados,73733
7,Huelva,Curados,57248
8,Huelva,Fallecidos,443


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  73474.0


/tmp/ipykernel_29155/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  21438.0


/tmp/ipykernel_29155/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_29155/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_29155/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_29155/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 273 personas en los últimos 7 días 

Un positivo PCR cada 87 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,73474.0,5586.0,1833.0,514763.0,356.086199,1085.159578,369.0
Huelva-Costa,42373.0,2817.0,889.0,290301.0,306.233874,970.372131,197.0
Sierra de Huelva-Andévalo Central,9872.0,1682.0,600.0,67400.0,890.207715,2495.548961,120.0
Huelva (capital),21438.0,1632.0,523.0,142538.0,366.919699,1144.957836,113.0
Condado-Campiña,20758.0,1078.0,342.0,157062.0,217.748405,686.353160,52.0
Valverde del Camino,2206.0,390.0,166.0,12714.0,1305.647318,3067.484663,28.0
Cortegana,758.0,163.0,59.0,4635.0,1272.923409,3516.720604,18.0
Cartaya,3141.0,174.0,49.0,20314.0,241.212957,856.552132,18.0
Aljaraque,2914.0,218.0,72.0,21706.0,331.705519,1004.330600,16.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Rosal de la Frontera,283.0,74.0,35.0,1682.0,2080.856124,4399.524376,11.0
Cortegana,758.0,163.0,59.0,4635.0,1272.923409,3516.720604,18.0
Valdelarco,14.0,8.0,1.0,236.0,423.728814,3389.830508,NaN
Cerro de Andévalo (El),468.0,78.0,16.0,2320.0,689.655172,3362.068966,5.0
Linares de la Sierra,24.0,9.0,3.0,268.0,1119.402985,3358.208955,0.0
Nerva,720.0,167.0,69.0,5127.0,1345.816267,3257.265457,8.0
Jabugo,314.0,72.0,20.0,2247.0,890.075656,3204.272363,6.0
Marines (Los),44.0,13.0,4.0,410.0,975.609756,3170.731707,0.0
Valverde del Camino,2206.0,390.0,166.0,12714.0,1305.647318,3067.484663,28.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Manzanilla,261.0,16.0,1.0,2152.0,46.468401,743.494424,0.0,0.062500
Paterna del Campo,408.0,13.0,1.0,3478.0,28.752156,373.778033,0.0,0.076923
Alájar,71.0,10.0,1.0,814.0,122.850123,1228.501229,0.0,0.100000
Almendro (El),89.0,9.0,1.0,835.0,119.760479,1077.844311,0.0,0.111111
Bonares,624.0,33.0,4.0,6075.0,65.843621,543.209877,0.0,0.121212
Valdelarco,14.0,8.0,1.0,236.0,423.728814,3389.830508,0.0,0.125000
Niebla,611.0,39.0,5.0,4158.0,120.250120,937.950938,0.0,0.128205
Corteconcepción,79.0,7.0,1.0,571.0,175.131349,1225.919440,0.0,0.142857
San Silvestre de Guzmán,89.0,6.0,1.0,653.0,153.139357,918.836141,1.0,0.166667
